In [1]:
import numpy as np
import pyro
import pyro.infer
import pyro.optim
import pyro.distributions as dist
import torch
from torch.utils.data import TensorDataset, DataLoader
from torch.autograd import Variable

def make_data(n_data=1000):
    """ Sample n_data datapoints from a 1-d mixture of Gaussians. """
    p = [.5, .5]
    means = np.random.choice([-2, 4], n_data, p=p)
    x = torch.Tensor(np.random.randn(n_data) + means).view(n_data, 1)
    # Use TensorDataset, but only make use of the 'data_tensor' param.
    return DataLoader(TensorDataset(data_tensor=x, target_tensor=x),
        batch_size=n_data, shuffle=True)

def model(data):
    x = Variable(data[0]).squeeze()
    n_data = x.size()[0]

    # Define p_{theta}(z).
    mu = pyro.param('mu', Variable(torch.Tensor([-2, 4]), requires_grad=True))
    sigma = Variable(torch.ones(n_data)) # Constant variances.
    ps = Variable(torch.Tensor([.5, .5])) # Constant component probabilities.
    zs = pyro.sample('zs', dist.categorical, ps, batch_size=n_data)

    # Define p_{theta}(x | z).
    comp_ids = torch.max(zs, 1)[1] # Choose a component for each datapoint.
    mus = mu[comp_ids] # Assign the appropriate mean to each datapoint.
    pyro.observe('obs', dist.normal, x, mus, sigma) # Condition on data.

    return mu.data.numpy() # We will monitor mu as training progresses.

def guide(data):
    n_data = data[0].size()[0]

    # Sample q(z) from a categorical distribution.
    ps = Variable(torch.Tensor([.5, .5]))
    pyro.sample('zs', dist.categorical, ps, batch_size=n_data)


svi = pyro.infer.SVI(model=model,
                     guide=guide,
                     optim=pyro.optim.Adam({'lr': 0.1}),
                     loss='ELBO')

dataloader = make_data()
for t in range(100):
    losses = []
    for batch in dataloader:
        losses.append(svi.step(batch))
    mus = model(batch)
    print('err={}. mus={}'.format(np.mean(losses), mus))

TypeError: __init__() got an unexpected keyword argument 'data_tensor'